# 解析 ONNX

## 1. ONNX API 解析

In [3]:
import onnx

# Load the ONNX model
model = onnx.load("./model/linear.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %data[FLOAT, 3x1]
) initializers (
  %linear.bias[FLOAT, 1]
  %linear.weight[FLOAT, 1x1]
) {
  %output1 = Gemm[alpha = 1, beta = 1, transB = 1](%data, %linear.weight, %linear.bias)
  return %output1
}


In [4]:
model.graph.output 

[name: "output1"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 1
      }
    }
  }
}
]

## 2. Python 解析 ONNX 模型
載入 ONNX 模型之後可以得到一個 ModelProto，其內容包含了一些版本訊息以及最重要的重要 GraphProto。
- node 為當前模型的所有結點
- input 為模型中所有的輸入節點
- output 為模型所有的輸出節點
- initializer 為模型權重

[參考](https://blog.51cto.com/u_15357586/5139273)

In [1]:
import onnx
import numpy as np

def onnx_datatype_to_npType(data_type):
    if data_type == 1:
        return np.float32
    else:
        raise TypeError("don't support data type")


def parser_initializer(initializer):
    name = initializer.name
    print(f"initializer name: {name}")

    dims = initializer.dims
    shape = [x for x in dims]
    print(f"initializer with shape:{shape}")

    dtype = initializer.data_type
    print(f"initializer with type: {onnx_datatype_to_npType(dtype)} ")
    
    # print tenth buffer
    weights = np.frombuffer(initializer.raw_data, dtype=onnx_datatype_to_npType(dtype))
    print(f"initializer first 10 wights:{weights[:10]}")



def parser_tensor(tensor, use='normal'):
    name = tensor.name
    print(f"{use} tensor name: {name}")

    data_type = tensor.type.tensor_type.elem_type
    print(f"{use} tensor data type: {data_type}")

    dims = tensor.type.tensor_type.shape.dim
    shape = []
    for i, dim in enumerate(dims):
        shape.append(dim.dim_value)
    print(f"{use} tensor with shape:{shape} ")


def parser_node(node):
    def attri_value(attri):
        if attri.type == 1:
            return attri.i
        elif attri.type == 7:
            return list(attri.ints)
        
    name = node.name
    print(f"node name:{name}")

    opType = node.op_type
    print(f"node op type:{opType}")

    inputs = list(node.input)
    print(f"node with {len(inputs)} inputs:{inputs}")

    outputs = list(node.output)
    print(f"node with {len(outputs)} outputs:{outputs}")
    
    attributes = node.attribute
    for attri in attributes:
        name = attri.name
        value = attri_value(attri)
        print(f"{name} with value:{value}")


def parser_info(onnx_model):
    ir_version = onnx_model.ir_version
    producer_name = onnx_model.producer_name
    producer_version = onnx_model.producer_version
    for info in [ir_version, producer_name, producer_version]:
        print("onnx model with info:{}".format(info))

def parser_inputs(onnx_graph):
    inputs = onnx_graph.input
    for input in inputs:
        parser_tensor(input, 'input')

def parser_outputs(onnx_graph):
    outputs = onnx_graph.output
    for output in outputs:
        parser_tensor(output, 'output')

def parser_graph_initializers(onnx_graph):
    initializers = onnx_graph.initializer
    for initializer in initializers:
        parser_initializer(initializer)


def parser_graph_nodes(onnx_graph):
    nodes = onnx_graph.node
    for node in nodes:
        parser_node(node)
        t = 1

def onnx_parser():
    model_path = './model/tf_model.onnx'
    model = onnx.load(model_path)

    # 0.
    parser_info(model)
    graph = model.graph
    # 1.
    parser_inputs(graph)
    # 2. 
    parser_outputs(graph)
    # 3.
    parser_graph_initializers(graph)
    # 4. 
    parser_graph_nodes(graph)

if __name__ == '__main__':
    onnx_parser()

onnx model with info:7
onnx model with info:tf2onnx
onnx model with info:1.12.0 a58786
input tensor name: tensor_input
input tensor data type: 1
input tensor with shape:[0, 4] 
output tensor name: tensor_output
output tensor data type: 1
output tensor with shape:[0, 3] 
initializer name: sequential/tensor_output/MatMul/ReadVariableOp:0
initializer with shape:[16, 3]
initializer with type: <class 'numpy.float32'> 
initializer first 10 wights:[ 0.17284542  0.04914844  0.34098202 -0.48509496 -0.1857123  -0.29634443
 -0.10584152  0.11752605 -0.13919416 -0.02178433]
initializer name: sequential/tensor_output/BiasAdd/ReadVariableOp:0
initializer with shape:[3]
initializer with type: <class 'numpy.float32'> 
initializer first 10 wights:[-0.13337871  0.37609738 -0.2843588 ]
initializer name: sequential/tensor/MatMul/ReadVariableOp:0
initializer with shape:[4, 8]
initializer with type: <class 'numpy.float32'> 
initializer first 10 wights:[-0.5601566  -0.6473129  -0.6140953  -0.2900776  -0.22043